## Create Images

In [ ]:
import os
import numpy as np
from os import path
from PIL import Image as PImage

In [ ]:
PATH_IMGS = path.join("..", "imgs")
PATH_MAPS_IN = path.join(PATH_IMGS, "00-map")
PATH_MASKS_IN = path.join(PATH_IMGS, "01-mask")
PATH_MAPS_OUT = path.join(PATH_IMGS, "02-sub-map")
PATH_MASKS_OUT = path.join(PATH_IMGS, "02-sub-mask")

image_filenames = [f for f in os.listdir(PATH_MAPS_IN) if f.endswith(".jpg")]
label_filenames = [f for f in os.listdir(PATH_MASKS_IN) if f.endswith(".jpg")]

In [ ]:
def create_mask(img, label):
    imglnp = np.asarray(img.convert("L"))
    imglnpt = np.where(imglnp > 128, label, 0)
    return PImage.fromarray(np.uint8(imglnpt))


def save_subimages(img, size, out_path):
    iw, ih = img.width, img.height
    num_cols, num_rows = (iw // size) + 1, (ih // size) + 1
    step_w, step_h = ((iw - size) // (num_cols - 1)), ((ih - size) // (num_rows - 1))

    for y in range(num_rows):
        top, bottom = y * step_h, y * step_h + size
        for x in range(num_cols):
            left, right = x * step_w, x * step_w + size
            sub_out_path = out_path.replace(".jpg", f"-{y:02}-{x:02}.jpg")
            img.crop((left, top, right, bottom)).save(sub_out_path)

In [ ]:
os.makedirs(PATH_MAPS_OUT, exist_ok=True)

for fn in sorted(image_filenames):
    in_path = path.join(PATH_MAPS_IN, fn)
    out_path = path.join(PATH_MAPS_OUT, fn)
    with PImage.open(in_path) as img:
        save_subimages(img=img, size=512, out_path=out_path)

In [ ]:
os.makedirs(PATH_MASKS_OUT, exist_ok=True)

for fn in sorted(label_filenames):
    in_path = path.join(PATH_MASKS_IN, fn)
    out_path = path.join(PATH_MASKS_OUT, fn)
    with PImage.open(in_path) as img:
        imgt = create_mask(img, label=1)
        save_subimages(img=imgt, size=512, out_path=out_path)

## Create Dataset

In [ ]:
import json
from datasets import Dataset, DatasetDict, Image

In [ ]:
PATH_IMGS = path.join("..", "imgs")
PATH_MAPS_IN = path.join(PATH_IMGS, "02-sub-map")
PATH_MASKS_IN = path.join(PATH_IMGS, "02-sub-mask")

In [ ]:
def create_dataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"pixel_values": sorted(image_paths),
                                "label": sorted(label_paths)})
    dataset = dataset.cast_column("pixel_values", Image())
    dataset = dataset.cast_column("label", Image())
    
    return dataset

In [ ]:
image_paths = sorted([path.join(PATH_MAPS_IN, f) for f in os.listdir(PATH_MAPS_IN) if f.endswith(".jpg")])
label_paths = sorted([path.join(PATH_MASKS_IN, f) for f in os.listdir(PATH_MASKS_IN) if f.endswith(".jpg")])

In [ ]:
train_dataset = create_dataset(image_paths, label_paths)

dataset = DatasetDict({
    "train": train_dataset,
  }
)

In [ ]:
dataset.push_to_hub("thiagohersan/satellite-trees-dataset", private=True)


In [ ]:
id2label = {0: "unlabeled", 1: "tree"}
with open('id2label.json', 'w') as fp:
    json.dump(id2label, fp)